In [1]:
import torch
import tiktoken
import re

In [8]:
raw_text = ""

with open("rfc.txt", "r") as file:
    raw_text = file.read()

#print(len(raw_text))
#raw_text = raw_text[:100]

# split into word
raw_words = re.split(r'\s+|[.,!?:;\'"()\-]', raw_text)

words = []

voca_index = 0
voca_pos = []
voca = {"</w>": [voca_index, 0]}
voca_pos.append("</w>")
voca_index += 1

for w in raw_words:
    if w.strip():
        for c in w:
            if c in voca:
                voca[c][1] += 1
            else:
                voca[c] = [voca_index, 1]
                voca_pos.append(c)
                voca_index += 1
        voca["</w>"][1] += 1

        words.append(w)

#print(len(words))
#print(words[:10])

for i, (char, count) in enumerate(voca.items()):
    if i >= 10:
        break
    print(f"{char}: {count}")

print("voca: ", len(voca))

</w>: [0, 94460]
N: [1, 1110]
e: [2, 56334]
t: [3, 37991]
w: [4, 3485]
o: [5, 28926]
r: [6, 27428]
k: [7, 1151]
W: [8, 317]
i: [9, 28690]
voca:  83


In [9]:
words = [list(w) for w in words]
for l_w in words:
    l_w.append("</w>")

#print("words", words[:10])

encoded = []
for w in words:
    encoded.append([voca[c][0] for c in w])

print(words[:10])

#print 10 item in voca
for i, (char, count) in enumerate(voca.items()):
    if i >= 10:
        break
    print(f"{char}: {count}")

[['N', 'e', 't', 'w', 'o', 'r', 'k', '</w>'], ['W', 'o', 'r', 'k', 'i', 'n', 'g', '</w>'], ['G', 'r', 'o', 'u', 'p', '</w>'], ['J', '</w>'], ['R', 'o', 's', 'e', 'n', 'b', 'e', 'r', 'g', '</w>'], ['R', 'e', 'q', 'u', 'e', 's', 't', '</w>'], ['f', 'o', 'r', '</w>'], ['C', 'o', 'm', 'm', 'e', 'n', 't', 's', '</w>'], ['3', '2', '6', '1', '</w>'], ['d', 'y', 'n', 'a', 'm', 'i', 'c', 's', 'o', 'f', 't', '</w>']]
</w>: [0, 94460]
N: [1, 1110]
e: [2, 56334]
t: [3, 37991]
w: [4, 3485]
o: [5, 28926]
r: [6, 27428]
k: [7, 1151]
W: [8, 317]
i: [9, 28690]


In [10]:
max_voca_size = 10000
while len(voca) < max_voca_size:
    pair_count = {}
    max_pair = 0
    firstPart, secondPart = -1, -1

    # Count the frequency of each pair of characters
    for w in encoded:
        for i in range(len(w) - 1):
            pair = (w[i], w[i+1])
            if pair in pair_count:
                pair_count[pair] += 1
            else:
                pair_count[pair] = 1

            if pair_count[pair] > max_pair:
                max_pair = pair_count[pair]
                firstPart = w[i]
                secondPart = w[i+1]
    
    # print(max_pair)
    #print(firstPart, secondPart)
    if firstPart == -1 or secondPart == -1 or max_pair < 5:
        print("No more pair to merge")
        break

    # Merge the most frequent pair
    newpair = voca_index
    voca_index += 1
    merged = voca_pos[firstPart] + voca_pos[secondPart]
    voca_pos.append(merged)

    # Update the encoded words
    voca[merged] = [voca_index, max_pair]
    voca[voca_pos[firstPart]][1] -= max_pair
    voca[voca_pos[secondPart]][1] -= max_pair
    if voca[voca_pos[firstPart]][1] == 0:
        del voca[voca_pos[firstPart]]
    if voca[voca_pos[secondPart]][1] == 0:
        del voca[voca_pos[secondPart]]

    #print(len(voca))
    #print(voca[merged])
    #print(merged)

    #replace the pair in the encoded words
    isMerged = False
    for w in encoded:
        i = 0
        while i < len(w) - 1:
            if w[i] == firstPart and w[i+1] == secondPart:
                w[i] = newpair
                w.pop(i+1)
                isMerged = True
            i += 1
    #print("merge status", isMerged)
    #print(len(encoded))        
    
print(len(voca))

# print 10 last item in voca_pos
print(voca_pos[-10:])

print(encoded[:10])

print("word", words[:10])


No more pair to merge
3535
['Ber', 'School', 'Schooler</w>', 'abs</w>', 'Copyright</w>', 'kind</w>', 'ual</w>', 'PA', 'Soci', 'Society</w>']
[[1, 195, 824], [3117, 1403], [12, 393, 205], [1594], [357], [351], [161], [1072, 1536], [366], [3658]]
word [['N', 'e', 't', 'w', 'o', 'r', 'k', '</w>'], ['W', 'o', 'r', 'k', 'i', 'n', 'g', '</w>'], ['G', 'r', 'o', 'u', 'p', '</w>'], ['J', '</w>'], ['R', 'o', 's', 'e', 'n', 'b', 'e', 'r', 'g', '</w>'], ['R', 'e', 'q', 'u', 'e', 's', 't', '</w>'], ['f', 'o', 'r', '</w>'], ['C', 'o', 'm', 'm', 'e', 'n', 't', 's', '</w>'], ['3', '2', '6', '1', '</w>'], ['d', 'y', 'n', 'a', 'm', 'i', 'c', 's', 'o', 'f', 't', '</w>']]


In [11]:
print(voca["knowledg"])
print(voca["e</w>"])

[2735, 3]
[84, 65]


In [12]:
decode = []

for w in encoded:
    decode.append([voca_pos[c] for c in w])

print(decode[:10])

[['N', 'et', 'work</w>'], ['Wor', 'king</w>'], ['G', 'rou', 'p</w>'], ['J</w>'], ['Rosenberg</w>'], ['Request</w>'], ['for</w>'], ['Com', 'ments</w>'], ['3261</w>'], ['dynamicsoft</w>']]
